# 1) Données

In [14]:
import pandas as pd
import numpy as np
import csv
import os

# calcul de la position dans le roulement d'un poste en fonction de la semaine
def rFunction(s,p):
    return ((p-s)%24)

# calcul du numéro de semaine correspondant au jour j
def sFunction(j):
    return j//7

# calcul du jour non travaillé pour une semaine donnée d'un opérateur à 80%
# en fonction du jour non travaillé la première semaine
def delta_i(s,d_i):
    return (d_i+s)%5

# operateurs + competences
dataOperateurs = pd.read_csv('./data/operateurs.csv',sep = ";" )

cardO = len(dataOperateurs.axes[0])
colsDataOperateurs = len(dataOperateurs.axes[1])
nomsOperateurs = dataOperateurs.iloc[0:cardO,0:3]
competencesOperateurs = dataOperateurs.iloc[0:cardO,3:(colsDataOperateurs+1)]

# postes (intitulés + rho)
dataPostes = pd.read_csv('./data/postes.csv',sep = ";" )
cardP = len(dataPostes.axes[0])
nomsPostes = dataPostes.iloc[0:cardO,1]
rho = [0,1,1,1,0,1,1,1,0,1,1,1,0,1,1,1,0,1,1,1,0,1,1,1]

# le nombre de semaines <=> nombre de postes <=> nombre d'opérateurs
#cardS = cardO
cardS = 4
cardJ = 7*cardS
cardR = cardO

# kappa_ip (opérateur possède la compétence pour le poste)
#kappa = pd.read_csv('./data/kappa_ip.csv',sep = ";" )
kappa = [
[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],
[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],
[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1],
[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],
[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],
[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1],
[1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,1,0,1,1,1,0,0,0],
[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],
[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1],
[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],
[1,1,1,1,1,1,0,0,1,1,1,0,1,1,1,1,1,1,0,1,1,1,1,1],
[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],
[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],
[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],
[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],
[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],
[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1],
[1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,1,0,1,1,1,0,0,0],
[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],
[1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,1,0,1,1,1,0,0,0],
[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1],
[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],
[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],
[1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,1,0,1,1,1,0,0,0]]

# sigma_pp' (postes sur le même créneau horaire)
#sigma = pd.read_csv('./data/sigma_pp.csv',sep = ";" )
sigma= [[1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
[0,1,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,1],
[0,0,1,0,0,1,0,1,0,0,1,0,0,1,0,1,0,1,0,1,0,0,1,0],
[0,1,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,1],
[0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
[0,0,1,0,0,1,0,1,0,0,1,0,0,1,0,1,0,1,0,1,0,0,1,0],
[0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0],
[0,0,1,0,0,1,0,1,0,0,1,0,0,1,0,1,0,1,0,1,0,0,1,0],
[0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
[0,1,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,1],
[0,0,1,0,0,1,0,1,0,0,1,0,0,1,0,1,0,1,0,1,0,0,1,0],
[0,1,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,1],
[0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0],
[0,0,1,0,0,1,0,1,0,0,1,0,0,1,0,1,0,1,0,1,0,0,1,0],
[0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0],
[0,0,1,0,0,1,0,1,0,0,1,0,0,1,0,1,0,1,0,1,0,0,1,0],
[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0],
[0,0,1,0,0,1,0,1,0,0,1,0,0,1,0,1,0,1,0,1,0,0,1,0],
[0,1,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,1],
[0,0,1,0,0,1,0,1,0,0,1,0,0,1,0,1,0,1,0,1,0,0,1,0],
[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0],
[0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0],
[0,0,1,0,0,1,0,1,0,0,1,0,0,1,0,1,0,1,0,1,0,0,1,0],
[0,1,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,1]]

# delta_ij (est-ce qu'une personne à 80% travaille le jour j)
#delta = pd.read_csv('./data/delta_ij.csv',sep = ";" )
# d[i] représente le jour non travaillé de l'opérateur i la première semaine, d[i]=-1 si i est à temps plein
d = [3,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1]
delta = np.full((cardO, cardJ), 1)
for i in range(0,cardO):
    if d[i] != -1:
        d_i = d[i]
        for s in range(0, cardS):
            d_is = delta_i(s,d_i) #rang du jour non travaillé la semaine s
            j_s = 7*s + d_is      #numéro du jour non travaillé
            delta[i][j_s] = 0

# o_j (jour ouvrés)
o = np.zeros(cardJ)
for j in range(0,len(o)):
    if (j%7 <= 4):
        o[j] = 1

# H_s (jours de chaque semaine)
H = []
for i in range(0,cardS):
    H.append(np.zeros(7))
for i in range(0,cardS):
    for j in range(0,7):
        H[i][j]=(i*7 + j)

#print(nomsOperateurs)
#print(competencesOperateurs)
#print(nomsPostes)
#print(rho)
#print(kappa)
#print(sigma)
print(delta)
#print(o)
#print(H[1])


[[1 1 1 0 1 1 1 1 1 1 1 0 1 1 0 1 1 1 1 1 1 1 0 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 

# 2) Phase 1 : Modèle mathématique

## 2.1) Variables

In [2]:
from pulp import *

# x_ijp = 1 si l'opérateur prend en charge le poste p le jour j


# y_ir = 1 si l'opérateur i est position r dans le roulement
y = {} 
for i in range(0,cardO):
    for r in range(0,cardR):    
        y[i,r]=LpVariable(f"y({i,r})", cat=LpBinary)
        
# z_isp = 1 si l'opérateur est affecté au poste p la semaine s
z = {} 
for i in range(0,cardO):
    for s in range(0,cardS): 
        for p in range(0,cardP): 
            z[i,s,p]=LpVariable(f"z({i,s,p})", cat=LpBinary)

# u_is = 1 si l'opérateur i est affecté à un poste différent de celui prévu la semaine s
u = {} 
for i in range(0,cardO):
    for s in range(0,cardS):    
        u[i,s]=LpVariable(f"u({i,s})", cat=LpBinary)

print("nb de variables y_ir  = ", len(y))
print("nb de variables z_isp = ", len(z))
print("nb de variables u_ir  = ", len(u))
print()
print("nb total de variables = ", len(y)+len(z)+len(u))

nb de variables y_ir  =  576
nb de variables z_isp =  2304
nb de variables u_ir  =  96

nb total de variables =  2976


## 2.2) Création du modèle avec PuLP

In [3]:
prob = LpProblem("ModelPhaseUn", LpMinimize)

prob += sum(u[i,s] for i in range(0,cardO) for s in range(0,cardS)), "obj"

## 2.3) Contraintes

In [4]:
#contrainte 2 Un opérateur ne peut être affecté à un poste une semaine que s'il en possède la compétence

for i in range(0,cardO):
    for s in range(0,cardS):
        for p in range(0,cardP):
            prob += (z[i,s,p] <= kappa[i][p], f"C2_{i,s,p}")

#contrainte 7 Chaque semaine, chaque opérateur est affecté à un poste

for i in range(0,cardO):
    for s in range(0,cardS):
        prob += sum(z[i,s,p] for p in range(0,cardP)) == 1, f"C6_{i,s}"

#contrainte 8 Chaque semaine, chaque poste est assuré par un opérateur

for p in range(0,cardP):
    for s in range(0,cardS):
        prob += sum(z[i,s,p] for i in range(0,cardO)) == 1, f"C7_{s,p}"

#contrainte 9 Chaque opérateur est affecté à une position dans le roulement

for i in range(0,cardO):
    prob += sum(y[i,r] for r in range(0,cardR)) == 1, f"C8_{i,r}"

#contrainte 10 Chaque position dans le roulement est associée à un opérateur

for r in range(0,cardR):
    prob += sum(y[i,r] for i in range(0,cardO)) == 1, f"C9_{i,r}"

#contrainte 11 Un opérateur peut changer de poste si l'horaire de ce dernier est compatible

for i in range(0,cardO):
    for s in range(0,cardS):
        for p in range(0,cardP):
            prob += z[i,s,p] <= sum(sigma[p][p2]*y[i,rFunction(s,p2)] for p2 in range(0,cardP)), f"C10_{i,s,p}"

#contrainte 12 Lorsqu'un jour un opérateur est affecté à un poste A alors qu'il est affecté à un poste B pour cette semaine, cela augmente son score d'insatisfaction
for i in range(0,cardO):
    for s in range(0,cardS):
        for p in range(0,cardP):
            prob += u[i,s] >= (kappa[i][p]*rho[p]*y[i,rFunction(s,p)] - z[i,s,p]), f"C11_{i,s,p}"


## 2.4) Résolution du problème

In [5]:
import time

prob.writeLP("Modeles/ModelPhase1.lp")

start = time.time() 
prob.solve(CPLEX_PY())
end = time.time()

duree = end - start
print()
print("Temps de résolution : " + str(duree))
print("Valeur objectif : ", value(prob.objective))

Version identifier: 22.1.0.0 | 2022-03-09 | 1a383f8ce
CPXPARAM_Read_DataCheck                          1
Tried aggregator 1 time.
MIP Presolve eliminated 3144 rows and 132 columns.
Reduced MIP has 4008 rows, 2844 columns, and 24228 nonzeros.
Reduced MIP has 2844 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.03 sec. (12.95 ticks)
Found incumbent of value 96.000000 after 0.05 sec. (30.65 ticks)
Probing changed sense of 576 constraints.
Probing time = 0.05 sec. (11.55 ticks)
Tried aggregator 1 time.
MIP Presolve eliminated 600 rows and 576 columns.
Reduced MIP has 3408 rows, 2268 columns, and 22500 nonzeros.
Reduced MIP has 2268 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.02 sec. (17.38 ticks)
Probing time = 0.02 sec. (3.33 ticks)
Tried aggregator 1 time.
Detecting symmetries...
Reduced MIP has 3408 rows, 2268 columns, and 22500 nonzeros.
Reduced MIP has 2268 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.03 sec. (16.37 ticks)
P

# 3) Phase 2 : Modèle mathématique

## 3.1) Variables

In [6]:
x2 = {} 
for i in range(0,cardO):
    for j in range(0,cardJ): 
        for p in range(0,cardP): 
            x2[i,j,p]=LpVariable(f"x({i,j,p})", cat=LpBinary)

u2 = {} 
for i in range(0,cardO):
    for s in range(0,cardS):    
        u2[i,s]=LpVariable(f"u({i,s})", cat=LpBinary)

## 3.2) Création du modèle avec PuLP

In [7]:
prob2 = LpProblem("ModelPhase2", LpMinimize)

prob2 += sum(u2[i,s] for i in range(0,cardO) for s in range(0,cardS)), "obj"

## 3.3) Contraintes

In [8]:
#contrainte 1 Un opérateur ne peut prendre un poste que s'il est compétent et disponible sur un jour ouvré

for i in range(0,cardO):
    for j in range(0,cardJ):
        for p in range(0,cardP):
            prob2 += (x2[i,j,p] <= delta[i][j]*kappa[i][p]*o[j], f"C1_{i,j,p}")


#contrainte 3 Chaque jour, l'opérateur est affecté à son poste de la semaine (s'il n'est pas rouleur)
for i in range(0,cardO):
    for s in range(0,cardS):
        for j in H[s]:
            for p in range(0,cardP):
                prob2 += x2[i,j,p] <= z[i,s,p].varValue + sum((1 - rho[p2])*z[i,s,p2].varValue for p2 in range(0,cardP)), f"C3_{i,j,p}"

#contrainte 4 Chaque jour, un opérateur est affecté à au plus un poste

for i in range(0,cardO):
    for j in range(0,cardJ):
        if o[j] == 1 and delta[i][j] == 1:
            prob2 += sum(x2[i,j,p] for p in range(0,cardP)) == 1, f"C4_{i,j,p}"

#contrainte 5 Chaque jour, chaque poste est assuré par un opérateur
for p in range(0,cardP):
    for j in range(0,cardJ):
        if o[j] == 1 and rho[p] == 1:
            prob2 += sum(x2[i,j,p] for i in range(0,cardO)) == 1, f"C5_{i,j,p}"
            
#contrainte 6 Chaque jour, un poste roulant est occupé par au plus une personne
for p in range(0,cardP):
    for j in range(0,cardJ):
        if o[j] == 1 and rho[p] == 0:
            prob2 += sum(x2[i,j,p] for i in range(0,cardO)) <= 1, f"C52_{i,j,p}"
            
for i in range(0,cardO):
    for s in range(0,cardS):
        prob2 += u2[i,s] == u[i,s].varValue, f"C6_{i,s}"

## 3.4) Résolution du problème

In [9]:
prob2.writeLP("Modeles/ModelPhase2.lp")

start = time.time() 
prob2.solve(CPLEX_PY())
end = time.time()

duree2 = end - start
print()
print("Temps de résolution : " + str(duree2))
print("Valeur objectif : ", value(prob2.objective))

print("Temps de résolution deux phases : " + str(duree + duree2))

Version identifier: 22.1.0.0 | 2022-03-09 | 1a383f8ce
CPXPARAM_Read_DataCheck                          1
Tried aggregator 1 time.
MIP Presolve eliminated 33066 rows and 15495 columns.
Reduced MIP has 242 rows, 729 columns, and 1458 nonzeros.
Reduced MIP has 729 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.03 sec. (15.86 ticks)
Found incumbent of value 0.000000 after 0.04 sec. (20.17 ticks)

Root node processing (before b&c):
  Real time             =    0.04 sec. (20.68 ticks)
Parallel b&c, 8 threads:
  Real time             =    0.00 sec. (0.00 ticks)
  Sync time (average)   =    0.00 sec.
  Wait time (average)   =    0.00 sec.
                          ------------
Total (root+branch&cut) =    0.04 sec. (20.68 ticks)
Cplex status= 101

Temps de résolution : 0.6187396049499512
Valeur objectif :  0.0
Temps de résolution deux phases : 1.3606622219085693


# 4) Affichage de la solution

In [10]:
print('Affectation aux roulements')
for i in range(0,24):
    for r in range(0,24):
        if(y[i,r].varValue == 1):
            print('y[',i,',',r,'] == 1')

print()
print('Affectations hebdomadaires')
for s in range(0,cardS):
    print('semaine',s)
    for i in range(0,cardO):
        for p in range(0,cardP):
            if z[i,s,p].varValue==1:
                print('semaine',s,',',i,'affecté au poste',p)
                
print('======================================================')

for p in range(0,cardP):
    print()
    if rho[p] == 1:
        print('poste',p,'non rouleur')
    else:
        print('poste',p,'rouleur')
    for j in range(0,cardJ):
        for i in range(0,cardO):
            if (x2[i,j,p].varValue == 1):
                print(i,'occupe le poste',p,'le jour',j)
                
print('======================================================')

                
for i in range(0,cardO):
    print()
    print('planning de l\'opérateur',i)
    for j in range(0,cardJ):
        for p in range(0,cardP):
            if (x2[i,j,p].varValue == 1):
                print('poste',p,'le jour',j)

Affectation aux roulements
y[ 0 , 16 ] == 1
y[ 1 , 12 ] == 1
y[ 2 , 15 ] == 1
y[ 3 , 18 ] == 1
y[ 4 , 14 ] == 1
y[ 5 , 19 ] == 1
y[ 6 , 0 ] == 1
y[ 7 , 7 ] == 1
y[ 8 , 22 ] == 1
y[ 9 , 4 ] == 1
y[ 10 , 5 ] == 1
y[ 11 , 23 ] == 1
y[ 12 , 9 ] == 1
y[ 13 , 17 ] == 1
y[ 14 , 2 ] == 1
y[ 15 , 1 ] == 1
y[ 16 , 21 ] == 1
y[ 17 , 10 ] == 1
y[ 18 , 20 ] == 1
y[ 19 , 8 ] == 1
y[ 20 , 3 ] == 1
y[ 21 , 6 ] == 1
y[ 22 , 11 ] == 1
y[ 23 , 13 ] == 1

Affectations hebdomadaires
semaine 0
semaine 0 , 0 affecté au poste 16
semaine 0 , 1 affecté au poste 12
semaine 0 , 2 affecté au poste 15
semaine 0 , 3 affecté au poste 18
semaine 0 , 4 affecté au poste 14
semaine 0 , 5 affecté au poste 19
semaine 0 , 6 affecté au poste 0
semaine 0 , 7 affecté au poste 7
semaine 0 , 8 affecté au poste 22
semaine 0 , 9 affecté au poste 4
semaine 0 , 10 affecté au poste 5
semaine 0 , 11 affecté au poste 23
semaine 0 , 12 affecté au poste 9
semaine 0 , 13 affecté au poste 17
semaine 0 , 14 affecté au poste 2
semaine 0 , 15

# 5) Extraction de la solution au format texte

In [11]:
with open("solutionsTexte/solutionModeleDeuxPhases", "w") as file_object:
        file_object.write(str(cardO) + "\n")           #cardO
        file_object.write(str(cardP) + "\n")           #cardP
        file_object.write(str(cardR) + "\n")           #cardR
        file_object.write(str(cardS) + "\n")           #cardS
        ligne = ""
        for p in range(0,cardP):
            ligne = ligne + str(rho[p])
        file_object.write(ligne + "\n")                #rho
        
        ligne = ""
        for j in range(0,cardJ):
            ligne = ligne + str(int(o[j]))
        file_object.write(ligne + "\n")                #o_j
        
        for i in range(0,cardO):
            ligne = ""
            for p in range(0,cardP):
                ligne = ligne + str(kappa[i][p])
            file_object.write(ligne + "\n")            #kappa_ip
            
        for p in range(0,cardP):
            ligne = ""
            for p2 in range(0,cardP):
                ligne = ligne + str(sigma[p][p2])
            file_object.write(ligne + "\n")            #sigma_pp'
            
        for i in range(0,cardO):
            ligne = ""
            for j in range(0,cardJ):
                ligne = ligne + str(delta[i][j])
            file_object.write(ligne + "\n")            #delta_ij
        
        for i in range(0,cardO):
            for j in range(0,cardJ):
                ligne = ""
                for p in range(0,cardP):
                    ligne = ligne + str(int(x2[i,j,p].varValue))
                file_object.write(ligne + "\n")        #x_ijp
        
        for i in range(0,cardO):
            ligne = ""
            for r in range(0,cardR):
                ligne = ligne + str(int(y[i,r].varValue))
            file_object.write(ligne + "\n")            #y_ir

        for i in range(0,cardO):
            for s in range(0,cardS):
                ligne = ""
                for p in range(0,cardP):
                    ligne = ligne + str(int(z[i,s,p].varValue))
                file_object.write(ligne + "\n")        #z_isp
        
        file_object.write(str(int(value(prob.objective))) + "\n")

# 6) Extraction de la solution au format CSV

In [12]:
import csv
with open('CSV/planningDeuxPhases.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    jours = [""]
    for s in range(0,cardS):
        jours.extend(["lundi","mardi","mercredi","jeudi","vendredi","samedi","dimanche"])
    writer.writerow(jours)
    
    dates = [""]
    for j in range(0,cardJ):
        dates.append("jour "+str(j))
    writer.writerow(dates)
    
    for i in range(0,cardO):
        planningPerso = [nomsOperateurs.iloc[i,1] + " " + nomsOperateurs.iloc[i,2]]
        for j in range(0,cardJ):
            found = False
            for p in range(0,cardP):
                if x2[i,j,p].varValue == 1:
                    found = True
                    planningPerso.append(nomsPostes.iloc[p])
            if not found:
                if o[j] == 0:
                    planningPerso.append("")
                else:
                    planningPerso.append("RA")
        writer.writerow(planningPerso)

In [13]:
print("Fin de programme")

Fin de programme
